In [1]:
%env CUDA_VISIBLE_DEVICES=MIG-768d9c1d-110f-52e2-b0a2-3252f78280f8
device="cuda:0"
import torch
from k_diffusion.models.image_transformer_v2 import (
    ImageTransformerDenoiserModelV2,
    GlobalAttentionSpec,
    NeighborhoodAttentionSpec
)
from collections import namedtuple

env: CUDA_VISIBLE_DEVICES=MIG-768d9c1d-110f-52e2-b0a2-3252f78280f8


In [2]:
LevelSpec = namedtuple('LevelSpec', ['width', 'depth', 'd_ff', 'self_attn', 'dropout'])
MappingSpec = namedtuple('MappingSpec', ['depth', 'width', 'd_ff', 'dropout'])

levels = [
    LevelSpec(width=128, depth=2, d_ff=512, self_attn=NeighborhoodAttentionSpec(d_head=64, kernel_size=7), dropout=0.1),
    LevelSpec(width=256, depth=2, d_ff=1024, self_attn=NeighborhoodAttentionSpec(d_head=64, kernel_size=7), dropout=0.1),
    LevelSpec(width=512, depth=2, d_ff=2048, self_attn=NeighborhoodAttentionSpec(d_head=64, kernel_size=7), dropout=0.1),
    LevelSpec(width=1024, depth=4, d_ff=4096, self_attn=GlobalAttentionSpec(d_head=64), dropout=0.1)
]
mapping = MappingSpec(depth=4, width=512, d_ff=2048, dropout=0.1)

# Initialize the model
model = ImageTransformerDenoiserModelV2(
    levels=levels,
    mapping=mapping,
    in_channels=6,         # 3 channels for noisy image + 3 for lossy reconstruction
    out_channels=3,        # Output is the predicted clean RGB image
    patch_size=(4,4),          # Patch size for tokenization
    num_classes=0,         # No class conditioning
    mapping_cond_dim=0     # No additional mapping conditioning
).to(device)

In [ ]:
x = torch.randn(2,6,512,512).to(device)
sigma = torch.tensor(1.0).to(device)
with torch.no_grad():
    y = model(x,sigma=sigma)

/home/dgj335/.local/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
